# Generate Experiments

In [1]:
graph_base_dir_dean="/nvmeSpace/fuchs/deans-datasets/"
graph_base_dir_others="/nvmeSpace/fuchs/graphalytics-datasets/"
graph_logs_base_dir = "/nvmeSpace/fuchs/graphlogs/"

csr_lcc="csr4-lcc"
csr="csr"
csr_numa_lcc = "csr3-lcc-numa"

# Microexperiments
vector_al = "vector_al.3"
sorted_vector_al = "sorted_vector_al.6"
robin_hood_sorted_vector_al = "robin_hood_sorted_vector_al.3"
tree_sorted_vector_al = "tree_sorted_vector_al.3"
robin_hood_al = "robin_hood_al.3"
edgeiter_sorted_vector_al = "edgeiter_sorted_vector_al.3"

snn_csr_lcc = "single-numa-node-csr3-lcc"
snn_sorted_vector_al = "single-numa-node-sorted-vector_al.4"

csr_mb = "mb-csr.8"
vector_csr_mb = "vector-mb-csr.4"

# Competitors
llama="llama8-ref"
stinger="stinger7-ref"
teseo_lcc="teseo-lcc.12"
teseo_lcc_dv="teseo-lcc-dv.12b"
livegraph="livegraph3_ro"
sortledton="sortledton.3"
graphone="g1_v6-ref-ignore-build"

deans_al = "baseline_v3"

graph500_22 = "graph500-22"
graph500_24 = "graph500-24"
graph500_26 = "graph500-26"
uniform_24 = "uniform-24"
uniform_26 = "uniform-26"
dota_league = "dota-league"
friendster = "com-friendster"

library_2_write_threads={
    llama: 16,
    livegraph: 20,
    stinger: 56,
    teseo_lcc: 56,
    teseo_lcc_dv: 56,
    sortledton: 56,
    graphone: 20,
    csr_lcc: 56,
    sortledton: 56,
    csr: 56,
    csr_numa_lcc: 56,
    sorted_vector_al: 56,
    robin_hood_sorted_vector_al: 56,
    tree_sorted_vector_al: 56,
    vector_al: 56,
    robin_hood_al: 56,
    edgeiter_sorted_vector_al: 56,
    snn_csr_lcc: 28,
    snn_sorted_vector_al: 28,
    deans_al: 1,
    csr_mb: 56
}

graph_2_graphlog = {
    graph500_24: graph_logs_base_dir + "graph500-24-1.0.graphlog",
    uniform_24: graph_logs_base_dir + "uniform-24-1.0.graphlog"
}

blacklist = {
    llama: [graph500_26, uniform_26, friendster],
    stinger: [graph500_26, uniform_26, friendster], # TODO fix?
    graphone: [friendster] # TODO fix
}

blacklist_algo = {
    (llama, graph500_24): "lcc",
    (graphone, friendster): "lcc"
}

blacklist_lib_algo = {
    robin_hood_al: "sssp"
}

In [2]:
not_deans_datasets = [friendster]

def get_graph(name, dense):
    dean = name not in not_deans_datasets
    if dean:
        d = graph_base_dir_dean
    else:
        d = graph_base_dir_others + name + "/"
    if dense:
        name += "-dense"
    d += name
    d += ".properties"
    return d

def get_graph_lib(library, name):
    if library == teseo_lcc_dv:
        return get_graph(name, True)
    else:
        return get_graph(name, False)

In [3]:
all_libraries=[csr_numa_lcc,teseo_lcc,teseo_lcc_dv,stinger,livegraph,llama,graphone,sortledton]
microbenchmarks=[csr_mb,sorted_vector_al,robin_hood_sorted_vector_al,tree_sorted_vector_al,vector_al,robin_hood_al,edgeiter_sorted_vector_al]
all_graphs=[dota_league, graph500_22, graph500_24, uniform_24, graph500_26, uniform_26, friendster]

libraries=[teseo_lcc_dv]
graphs=[friendster]

#writer_threads = [1,2,4,8,14,28,42,56]
writer_threads = [0]

analytics = True

block_sizes=[512]

algo_blacklist=""

run_updates = False
measure_memory = False
validate = False

single_numa_node = False

default_parameters = {
    "-u": "",
    "-R": 1 if validate else 5,
    #"-R": 5,
    "-d": "/nvmeSpace/fuchs/experimental_results/results.sqlite3"
}

if not analytics:
    default_parameters["-R"] = 0
    
if run_updates:
    if teseo_lcc_dv in libraries: libraries.remove(teseo_lcc_dv)
    if csr_lcc in libraries: libraries.remove(csr_lcc)

In [4]:
import copy

experiments = []
for g in graphs:
    for l in libraries:
        for wt in writer_threads:
            
            if single_numa_node and not "single-numa-node" in l:
                raise Exception("Cannot generate single numa node runs and normal runs in one go.")
                
            if l in blacklist and g in blacklist[l]:
                continue        

            parameters = copy.copy(default_parameters)
            parameters["-l"] = l
            parameters["-G"] = get_graph_lib(l, g)
            if wt == 0:
                parameters["-w"] = library_2_write_threads[l]
            else:
                parameters["-w"] = wt

            if l in [csr_lcc, csr, csr_numa_lcc, snn_csr_lcc]:
                parameters["--load"] = ""
            
            blacklist_algos = []
            if algo_blacklist:
                blacklist_algos.append(algo_blacklist)
            if (l, g) in blacklist_algo:
                blacklist_algos.append(blacklist_algo[(l, g)])
                
            if l in blacklist_lib_algo:
                blacklist_algos.append(blacklist_lib_algo[l])
                
            if blacklist_algos:
                parameters["--blacklist"] = ",".join(blacklist_algos)
            
                
            if l == llama:
                parameters["--build_frequency"] = "10s"
            
            if run_updates:
                parameters["--log"] = graph_2_graphlog[g]
                
                parameters["--aging_timeout"] = "10h"
                if measure_memory:
                    parameters["--aging_memfp"] = ""
                    parameters["--aging_memfp_physical"] = ""
                    parameters["--aging_release_memory"] = "false"
                    
                    if l == llama:
                        parameters["--aging_memfp_threshold"] = "230G"
            if validate:
                parameters["--validate"] = " "
            
            if l == sortledton:
                for bs in block_sizes:
                    parameters["--block_size"] = bs
                    ex = ""
                    for (k, v) in parameters.items():
                        ex += " " + k + " " + str(v)

                    experiments.append(ex)
            else:    
                ex = ""
                for (k, v) in parameters.items():
                    ex += " " + k + " " + str(v)
                    
            

                experiments.append(ex)
experiments = list(map(lambda e: "./gfe_driver " + e, experiments))
if not (run_updates or single_numa_node):
    experiments = list(map(lambda e: "timeout 8h " + e, experiments))
if single_numa_node:
    experiments = list(map(lambda e: "numactl -N 0 -m 0 " + e, experiments))
for e in experiments:
    print(e)

timeout 8h ./gfe_driver  -u  -R 5 -d /nvmeSpace/fuchs/experimental_results/results.sqlite3 -l teseo-lcc-dv.12b -G /nvmeSpace/fuchs/graphalytics-datasets/com-friendster/com-friendster-dense.properties -w 56
